In [1]:
import torch, os
from google.colab import drive
drive.mount('/content/drive')
if os.getcwd() != "/content/drive/My Drive/TRSA":
  os.chdir("drive/MyDrive/TRSA")

Mounted at /content/drive


In [2]:
!pip install transformers datasets


     |████████████████████████████████| 4.0 MB 7.1 MB/s 
     |████████████████████████████████| 325 kB 32.5 MB/s 
     |████████████████████████████████| 6.6 MB 34.8 MB/s 
     |████████████████████████████████| 895 kB 36.6 MB/s 
     |████████████████████████████████| 77 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 56.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.0 MB/s 
     |████████████████████████████████| 136 kB 51.3 MB/s 
     |████████████████████████████████| 212 kB 63.0 MB/s 
     |████████████████████████████████| 127 kB 65.0 MB/s 
     |████████████████████████████████| 144 kB 41.8 MB/s 
     |████████████████████████████████| 94 kB 3.6 MB/s 
     |████████████████████████████████| 271 kB 63.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime

In [ ]:
if len(sys.argv) < 3:
    print("Usage: python train_mlm.py model_name data/train_sentences.txt [data/dev_sentences.txt]")
    exit()

In [5]:
from transformers import logging

logging.set_verbosity_warning()

In [36]:
model_name = "dbmdz/bert-base-turkish-uncased"
per_device_train_batch_size = 64

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = True                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.70                 #Probability that a word is replaced by a [MASK] token

In [37]:
from transformers import BertTokenizerFast
from transformers import BertForSequenceClassification
from transformers import AutoModel, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

loading configuration file https://huggingface.co/dbmdz/bert-base-turkish-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1bba11007b0853a5f63dc091fc5dd36989ea017cb369f660839de9efcf15847e.d34b61340faaa3122b620c12ba54ab8b1016bfeb2ff97c1d5e80b86808783b7a
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-turkish-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/dbmdz/bert-base-turkish-uncased/resolve/main/vocab.txt from cache at /root/.cache/h

In [38]:
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

loading configuration file https://huggingface.co/dbmdz/bert-base-turkish-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1bba11007b0853a5f63dc091fc5dd36989ea017cb369f660839de9efcf15847e.d34b61340faaa3122b620c12ba54ab8b1016bfeb2ff97c1d5e80b86808783b7a
Model config BertConfig {
  "_name_or_path": "dbmdz/bert-base-turkish-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/dbmdz/bert-base-turkish-uncased/resolve/main/pytorch_model.bin from cache a

In [ ]:
import pandas as pd
data= pd.read_csv("TRSAv1.csv",encoding="utf8")
#data.sample(frac=1)
data
#data.to_csv("train_cumle.csv", encoding='utf-8', index=False)

,id,score,review
0,0,Positive,Kokusu çok güzel etkisi de efsane
1,1,Negative,hiç bir fark olmadı siyah nokta silmedi
2,2,Positive,şeffaf güzel bir krem elimi yumuşacık yaptı
3,3,Positive,Özellikle karma ve hassas cildiniz varsa La Ro...
4,4,Negative,çok iyi nemlendiriyor ama daha ucuz muadilleri...
...,...,...,...
149995,149995,Neutral,Suruldugunde hafif bir parlaklik veriyor. Yika...
149996,149996,Positive,Mükemmel nemlendirici hayraniyim bu nemlendiri...
149997,149997,Positive,Çok güzel fiyatı da gayet uygun
149998,149998,Neutral,Fiyat olarak iyi ama düzenli kullansamda geçir...


In [ ]:
data=data['review']
data

0                         Kokusu çok güzel etkisi de efsane
1                   hiç bir fark olmadı siyah nokta silmedi
2               şeffaf güzel bir krem elimi yumuşacık yaptı
3         Özellikle karma ve hassas cildiniz varsa La Ro...
4         çok iyi nemlendiriyor ama daha ucuz muadilleri...
                                ...                        
149995    Suruldugunde hafif bir parlaklik veriyor. Yika...
149996    Mükemmel nemlendirici hayraniyim bu nemlendiri...
149997                      Çok güzel fiyatı da gayet uygun
149998    Fiyat olarak iyi ama düzenli kullansamda geçir...
149999                    çok güzel, sağlıklı, temiz, ferah
Name: review, Length: 150000, dtype: object

In [ ]:
test=data.iloc[:30000]
train=data.iloc[30000:]

In [ ]:
train.to_csv("train_cumle.csv", encoding='utf-8', index=False)
test.to_csv("test_cumle.csv", encoding='utf-8', index=False)

In [20]:
##### Load our training datasets
train_path = "train_cumle.csv"

train_sentences = []
with open(train_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip()
        if len(line) >= 10:
            train_sentences.append(line)

print("Train sentences:", len(train_sentences))

Train sentences: 119779


In [21]:
test_sentences = []
#train_path = sys.argv[2]
test_path = "test_cumle.csv"


with open(test_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        #line = line.split()
        line = line.strip()
        if len(line) >= 10:
            test_sentences.append(line)

print("Test sentences:", len(test_sentences))

Test sentences: 29940


In [39]:
#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

In [40]:
train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(test_sentences, tokenizer, max_length, cache_tokenization=True) if len(test_sentences) > 0 else None

In [41]:
if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

In [42]:
output_dir = "TR70"
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

Using amp half precision backend


In [44]:
trainer.train()

model_path = output_dir
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 119779
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 5616


Step,Training Loss,Validation Loss
1000,4.793300,4.586816
2000,4.608600,4.502322
3000,4.516800,4.450770
4000,4.471300,4.413001
5000,4.425800,4.393879


***** Running Evaluation *****
  Num examples = 29940
  Batch size = 8
Saving model checkpoint to TR70/checkpoint-1000
Configuration saved in TR70/checkpoint-1000/config.json
Model weights saved in TR70/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 29940
  Batch size = 8
Saving model checkpoint to TR70/checkpoint-2000
Configuration saved in TR70/checkpoint-2000/config.json
Model weights saved in TR70/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [TR70/checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 29940
  Batch size = 8
Saving model checkpoint to TR70/checkpoint-3000
Configuration saved in TR70/checkpoint-3000/config.json
Model weights saved in TR70/checkpoint-3000/pytorch_model.bin
Deleting older checkpoint [TR70/checkpoint-2000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 29940
  Batch size = 8
Saving model checkpoint to TR70/checkpoint-4000
Configuration 